# First Order Logic with Tensors

This is an implemetation of [Towards a Formal Distributional Semantics: Simulating Logical Calculi with Tensors](https://arxiv.org/pdf/1304.5823.pdf) (Grefenstette, 2013) using `python` and `numpy`.

In [1]:
import LogicModel as m

## Build the Domain

The domain can be loaded as a `list` of elements.

In [2]:
my_domain = [
    "john", 
    "chris", 
    "tom", 
    "mary", 
    "bill"
]

## Build the Predicates

Predicates can be loaded as a `dict` with the `key` as the predicate name and the `value` as a `list` representing the elements for which this predicate is true.

### Unary Predicates

In [3]:
my_unary_preds = {
    "is_mathematician": ["john", "chris"],    # John and Chris are mathematicians, the others are not
    "is_single": ["bill"]                     # Bill is the only single person in the domain
}

### Binary Predicates

In this case, the elements of the `list` are a `tuple` representing the two elements involved in the binary predicate.

In [4]:
my_binary_preds = {
    "hates": [("tom", "chris"), ("tom", "john"), ("chris", "chris")],   # e.g. Tom hates Chris and John but no else
    "loves": [("mary", "john"), ("john", "john"), ("mary", "mary")]     # e.g. Mary loves John and herself and no one else
}

## Build the Model

In [5]:
my_model = m.LogicModel(
    listOfElements=my_domain,
    dictionaryOfUnaryPredicates=my_unary_preds,
    dictionaryOfBinaryPredicates=my_binary_preds
)

my_model.buildAll()

Here's what the matrix representing "is_mathematician" looks like where column represents an element in the domain.

In [6]:
 my_model.unaryPredicateMatrices["is_mathematician"]

array([[ 1.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  1.,  1.]])

Binary predicates require a tensor where each column still represents an element in the domain, but the rest requires a bit more work to interpret.

In [11]:
my_model.binaryPredicateTensors["loves"]

array([[[ 1.,  0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  0.,  0.]],

       [[ 0.,  1.,  1.,  0.,  1.],
        [ 1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  0.,  1.],
        [ 1.,  1.,  1.,  1.,  1.]]])

## Accessing Truth in the Model

Determining the truth of a predicate will output a `2-dimensional` array where: <br>
$[1, 0] = True$ <br>
$[0, 1] = False$ 

### "Tom is a mathematician"

In [7]:
tom_mathematician = my_model.unaryOp(predicate="is_mathematician", element="tom")
tom_mathematician

array([[ 0.],
       [ 1.]])

### "Mary loves John or Mary loves Chris"

In [8]:
mary_loves_john = my_model.binaryOp(predicate="loves", subjElement="mary", objElement="john")     # True
mary_loves_chris = my_model.binaryOp(predicate="loves", subjElement="mary", objElement="chris")   # False

mary_loves = my_model.orOp(
    truthValue1=mary_loves_john,
    truthValue2=mary_loves_chris
)

mary_loves

array([[ 1.],
       [ 0.]])

## Modeling Uncertainty in Truth Values

We can adjust the truth values of any predicate to allow for uncertainty.

### How confident are we about who is a mathematician?

In [19]:
my_model.updateUnaryPredicate(
    element="john", 
    predicate="is_mathematician", 
    prob=.99                            # We are 99% confident John is a mathematician
)
my_model.updateUnaryPredicate(element="chris", predicate="is_mathematician", prob=.7)
my_model.updateUnaryPredicate(element="bill", predicate="is_mathematician", prob=.08)

In [20]:
my_model.unaryPredicateMatrices["is_mathematician"]

array([[ 0.99,  0.7 ,  0.  ,  0.  ,  0.08],
       [ 0.01,  0.3 ,  1.  ,  1.  ,  0.92]])

In [21]:
my_model.unaryOp(predicate="is_mathematician", element="john")

array([[ 0.99],
       [ 0.01]])

### How confident are we about who loves whom?

In [31]:
my_model.updateBinaryPredicate(
    pair=("mary", "chris"),
    predicate="loves",
    prob=.6                        # We are only 60% confident that Mary loves Chris
)
my_model.updateBinaryPredicate(("mary", "john"), "loves", .05)

In [32]:
my_model.binaryPredicateTensors["loves"]

array([[[ 1.  ,  0.  ,  0.  ,  0.05,  0.  ],
        [ 0.  ,  0.  ,  0.  ,  0.6 ,  0.  ],
        [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
        [ 0.  ,  0.  ,  0.  ,  1.  ,  0.  ],
        [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ]],

       [[ 0.  ,  1.  ,  1.  ,  0.95,  1.  ],
        [ 1.  ,  1.  ,  1.  ,  0.4 ,  1.  ],
        [ 1.  ,  1.  ,  1.  ,  1.  ,  1.  ],
        [ 1.  ,  1.  ,  1.  ,  0.  ,  1.  ],
        [ 1.  ,  1.  ,  1.  ,  1.  ,  1.  ]]])

In [33]:
mary_loves_john_ = my_model.binaryOp(predicate="loves", subjElement="mary", objElement="john")     # 60% likely
mary_loves_chris_ = my_model.binaryOp(predicate="loves", subjElement="mary", objElement="chris")   # 5% likely

mary_loves_ = my_model.orOp(
    truthValue1=mary_loves_john_,
    truthValue2=mary_loves_chris_
)

mary_loves_

array([[ 0.62],
       [ 0.38]])